<a href="https://colab.research.google.com/github/charactr-platform/vocos/blob/main/notebooks/Bark%2BVocos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Audio Synthesis using Bark and Vocos

In this notebook, we use [Bark](https://github.com/suno-ai/bark) generative model to turn a text prompt into EnCodec audio tokens. These tokens then go through two decoders, EnCodec and Vocos, to reconstruct the audio waveform. Compare the results to discover the differences in audio quality and characteristics.

Make sure you have Bark and Vocos installed:

In [ ]:
!pip install git+https://github.com/suno-ai/bark.git
!pip install vocos

Download and load Bark models

In [ ]:
from bark import preload_models

preload_models()

Download and load Vocos.

In [ ]:
from vocos import Vocos
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocos = Vocos.from_pretrained("charactr/vocos-encodec-24khz").to(device)

We are going to reuse `text_to_semantic` from Bark API, but to reconstruct audio waveform with a custom vododer, we need to slightly redefine the API to return `fine_tokens`.

In [ ]:
from typing import Optional, Union, Dict

import numpy as np
from bark.generation import generate_coarse, generate_fine


def semantic_to_audio_tokens(
    semantic_tokens: np.ndarray,
    history_prompt: Optional[Union[Dict, str]] = None,
    temp: float = 0.7,
    silent: bool = False,
    output_full: bool = False,
):
    coarse_tokens = generate_coarse(
        semantic_tokens, history_prompt=history_prompt, temp=temp, silent=silent, use_kv_caching=True
    )
    fine_tokens = generate_fine(coarse_tokens, history_prompt=history_prompt, temp=0.5)

    if output_full:
        full_generation = {
            "semantic_prompt": semantic_tokens,
            "coarse_prompt": coarse_tokens,
            "fine_prompt": fine_tokens,
        }
        return full_generation
    return fine_tokens

Let's create a text prompt and generate audio tokens:

In [ ]:
from bark import text_to_semantic

history_prompt = None
text_prompt = "So, you've heard about neural vocoding? [laughs] We've been messing around with this new model called Vocos."
semantic_tokens = text_to_semantic(text_prompt, history_prompt=history_prompt, temp=0.7, silent=False,)
audio_tokens = semantic_to_audio_tokens(
    semantic_tokens, history_prompt=history_prompt, temp=0.7, silent=False, output_full=False,
)

Reconstruct audio waveform with EnCodec:

In [ ]:
from bark.generation import codec_decode
from IPython.display import Audio

encodec_output = codec_decode(audio_tokens)

import torchaudio
# Upsample to 44100 Hz for better reproduction on audio hardware
encodec_output = torchaudio.functional.resample(torch.from_numpy(encodec_output), orig_freq=24000, new_freq=44100)
Audio(encodec_output, rate=44100)

Reconstruct with Vocos:

In [ ]:
audio_tokens_torch = torch.from_numpy(audio_tokens).to(device)
features = vocos.codes_to_features(audio_tokens_torch)
vocos_output = vocos.decode(features, bandwidth_id=torch.tensor([2], device=device))  # 6 kbps
# Upsample to 44100 Hz for better reproduction on audio hardware
vocos_output = torchaudio.functional.resample(vocos_output, orig_freq=24000, new_freq=44100).cpu()
Audio(vocos_output.numpy(), rate=44100)

Optionally save to mp3 files:

In [ ]:
torchaudio.save("encodec.mp3", encodec_output[None, :], 44100, compression=128)
torchaudio.save("vocos.mp3", vocos_output, 44100, compression=128)